# Imports

In [1]:
!pip install llama-index
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-huggingface
!pip install --upgrade huggingface_hub

!pip install accelerate
!pip install bitsandbytes
!pip install transformers
!pip install peft
!pip install einops
!pip install safetensors
!pip install torch

# !pip install trulens-eval
!pip install litellm
!pip install git+https://github.com/piotrm0/trulens.git@piotrm/jsonify-stack-size-protect#subdirectory=trulens_eval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached 

In [1]:
import transformers
import torch
import warnings
warnings.filterwarnings('ignore')
import nest_asyncio
import numpy as np
import pandas as pd

from transformers import BitsAndBytesConfig, AutoTokenizer

# Load Dataset (Knowledge Base)

In [2]:
from llama_index.core import SimpleDirectoryReader, Document

documents = SimpleDirectoryReader(
    input_files=["eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

print("Type: ", type(documents), "\n")
print("Number of documents: ", len(documents), "\n")
print("Type of document: ", type(documents[0]))
print("Document example: \n", documents[10].text)

document = Document(text="\n\n".join([doc.text for doc in documents]))

Type:  <class 'list'> 

Number of documents:  41 

Type of document:  <class 'llama_index.core.schema.Document'>
Document example: 
 PAGE 11
The Best Way to Build 
a New Habit
One of my favorite books is BJ Fogg’s, Tiny Habits: The Small Changes That Change 
Everything. Fogg explains that the best way to build a new habit is to start small 
and succeed, rather than start  too big and fail. For example, rather than trying to 
exercise for 30 minutes a day, he recommends aspiring to do just one push-up, and 
doing it consistently.
This approach may be helpful to those of you who want to spend more time studying. 
If you start by holding yourself accountable for watching, say, 10 seconds of an 
educational video every day — and you do so consistently — the habit of studying daily 
will grow naturally. Even if you learn nothing in that 10 seconds, you’re establishing the 
habit of studying a little every day. On some days, maybe you’ll end up studying for an 
hour or longer.


# Sentence Window RAG

In [3]:
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings

# !huggingface-cli login --token "hf_token"

In [4]:
# LLM Configs
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
llm_name = "mistralai/Mistral-7B-Instruct-v0.1"
prompt_template = PromptTemplate("<s>[INST] {query_str} [/INST] </s>\n")
llm_configs = {
    "do_sample": True,
    "temperature": 0.1,
    "top_k": 5,
    "top_p": 0.95
}

# Set the LLM
llm = HuggingFaceLLM(
    model_name=llm_name,
    tokenizer_name=llm_name,
    query_wrapper_prompt=prompt_template,
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},
    generate_kwargs=llm_configs,
    device_map="auto",
)
Settings.llm = llm


from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# set the embed model
embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)
Settings.embed_model = embed_model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core import VectorStoreIndex, ServiceContext, load_index_from_storage, StorageContext
from llama_index.core.indices.postprocessor import MetadataReplacementPostProcessor, SentenceTransformerRerank
import os


def build_sentence_window_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    sentence_window_size=3,
    save_dir="sentence_index",
):
    # create the sentence window node parser w/ default settings
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=sentence_window_size,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    sentence_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model=embed_model,
        node_parser=node_parser,
    )
    if not os.path.exists(save_dir):
        sentence_index = VectorStoreIndex.from_documents(
            documents, service_context=sentence_context
        )
        sentence_index.storage_context.persist(persist_dir=save_dir)
    else:
        sentence_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=sentence_context,
        )

    return sentence_index


def get_sentence_window_query_engine(
    sentence_index, similarity_top_k=6, rerank_top_n=2
):
    # define postprocessors
    postproc = MetadataReplacementPostProcessor(target_metadata_key="window")
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )

    sentence_window_engine = sentence_index.as_query_engine(
        similarity_top_k=similarity_top_k, node_postprocessors=[postproc, rerank]
    )
    return sentence_window_engine

In [6]:
sentence_index_1 = build_sentence_window_index(
    documents,
    llm=llm,
    embed_model=embed_model,
    sentence_window_size=1,
    save_dir="sentence_index_1",
)

sentence_window_engine_1 = get_sentence_window_query_engine(
    sentence_index_1
)

# RAG Triad of Metrics

## Make evaluation set

In [7]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [8]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'What is the right AI job for me?']

## Instantiate a Provider

In [9]:
from trulens_eval import LiteLLM

nest_asyncio.apply()
provider = LiteLLM(model_engine='huggingface/'+llm_name)

## Feedback Functions

In [10]:
from trulens_eval import Feedback, TruLlama, Select

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

context_selection = TruLlama.select_source_nodes().node.text

f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons,
        name="Groundedness")
    .on(context_selection)
    .on_output()
)

feedbacks = [
        f_qa_relevance,
        f_qs_relevance,
        f_groundedness
    ]

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


## Evaluation

In [13]:
from trulens_eval import Tru
import litellm

tru = Tru()
tru.reset_database()

def run_evals(eval_questions, tru_recorder, query_engine):
    for question in eval_questions:
        with tru_recorder as recording:
            query_engine.query(question)

tru_recorder_1 = TruLlama(
    sentence_window_engine_1,
    app_id="Sentence Window Engine 1",
    feedbacks=feedbacks
)

litellm.set_verbose=True
run_evals(eval_questions, tru_recorder_1, sentence_window_engine_1)

Streaming output truncated to the last 5000 lines.
https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1 \
-H 'content-type: *****' \
-d '{'inputs': '<s>[INST] You are a INFORMATION OVERLAP classifier; providing the overlap of information between the source and statement.\n        Respond only as a number from 0 to 10 where 0 is no information overlap and 10 is all information is overlapping.\n        Never elaborate. [/INST]</s> [INST] SOURCE: If you’re leaving \na job, exit gracefully.  Give your employer ample notice, give your full effort through your last \nhour on the job, transition unfinished business as best you can, and leave in a way that honors \nthe responsibilities you were entrusted with.Finding the Right AI Job For You CHAPTER 9\n        \n        Hypothesis: 2.\n        \n        Please answer with the template below for all statement sentences:\n\n        Criteria: <Statement Sentence>, \n        Supporting Evidence: <Identify and describe the

In [15]:
records, feedback = tru.get_records_and_feedback(app_ids=[
    "Sentence Window Engine 1"
])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Sentence Window Engine 1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_2ce842e0f45ac92ab05b8f64d15fa074,"""What are the keys to building a career in AI?""","""\nThe keys to building a career in AI are lea...",-,"{""record_id"": ""record_hash_2ce842e0f45ac92ab05...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-06-09T15:30:21.871452"", ""...",2024-06-09T15:30:29.090933,[],[],[],7,0,0.0
1,Sentence Window Engine 1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_98863f46ef31934852e60f63f6540cbc,"""How can teamwork contribute to success in AI?""","""\nTeamwork can contribute to success in AI by...",-,"{""record_id"": ""record_hash_98863f46ef31934852e...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-06-09T15:30:29.649128"", ""...",2024-06-09T15:30:43.026806,[],[],NaN,13,0,0.0
2,Sentence Window Engine 1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_3ced2e253f2febc06db599f6084fc017,"""What is the importance of networking in AI?""","""\nThe importance of networking in AI cannot b...",-,"{""record_id"": ""record_hash_3ced2e253f2febc06db...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-06-09T15:30:43.590939"", ""...",2024-06-09T15:30:56.584248,NaN,NaN,NaN,12,0,0.0
3,Sentence Window Engine 1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_5cdb8574625f3706de652cdc2735a9e4,"""What are some good habits to develop for a su...","""\nSome good habits to develop for a successfu...",-,"{""record_id"": ""record_hash_5cdb8574625f3706de6...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-06-09T15:30:57.143568"", ""...",2024-06-09T15:31:07.206351,NaN,NaN,NaN,10,0,0.0
4,Sentence Window Engine 1,"{""tru_class_info"": {""name"": ""TruLlama"", ""modul...",RetrieverQueryEngine(llama_index.core.query_en...,record_hash_569e9b6571ab94cb606b1a06bd3ce4cb,"""How can altruism be beneficial in building a ...","""\nAltruism, or the practice of selflessness a...",-,"{""record_id"": ""record_hash_569e9b6571ab94cb606...","{""n_requests"": 0, ""n_successful_requests"": 0, ...","{""start_time"": ""2024-06-09T15:31:07.864677"", ""...",2024-06-09T15:31:32.944200,NaN,NaN,NaN,25,0,0.0


In [23]:
feedback


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers



[]


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers



In [24]:
pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output
0,"""What are the keys to building a career in AI?""","""\nThe keys to building a career in AI are learning foundational technical skills, working on projects, and finding a job, all of which is supported by being part of a community."""
1,"""How can teamwork contribute to success in AI?""","""\nTeamwork can contribute to success in AI by allowing individuals to collaborate, influence, and be influenced by others. This is particularly important when tackling large projects, as working in teams can lead to better outcomes than working individually. By leveraging the strengths and expertise of team members, individuals can gain new insights and perspectives, which can help them make more informed decisions and ultimately lead to greater success. Additionally, teamwork can help individuals build relationships and networks, which can be valuable in the competitive field of AI."""
2,"""What is the importance of networking in AI?""","""\nThe importance of networking in AI cannot be overstated. Building a strong professional network can help propel you forward in your career in AI. Having genuine friends in the field who you can count on for help and advice can be invaluable. In addition to networking, it can be helpful to think about building up a community. People you've met can provide valuable information and can also play an invaluable role by referring you to potential employers."""
3,"""What are some good habits to develop for a successful career?""","""\nSome good habits to develop for a successful career include developing good habits in eating, exercise, sleep, personal relationships, work, learning, and self-care. These habits help individuals move forward while staying healthy. Additionally, people who aim to lift others during every step of their own journey often achieve better outcomes for themselves."""
4,"""How can altruism be beneficial in building a career?""","""\nAltruism, or the practice of selflessness and putting others' needs before one's own, can be beneficial in building a career in several ways. Firstly, it can lead to better outcomes for oneself. People who aim to lift others during their journey often achieve better outcomes for themselves. This is because helping others can lead to increased motivation, a sense of purpose, and a greater sense of fulfillment, which can ultimately lead to greater success in one's career.\n\nSecondly, altruism can help to build strong relationships and networks. When you help others, you demonstrate your commitment to their success and build trust and credibility. This can lead to opportunities for collaboration, mentorship, and referrals, which can be valuable in building a career.\n\nFinally, altruism can help to create a positive impact in the world. By using one's skills and resources to help others, you can make a difference in people's lives and contribute to a greater good. This can be a powerful motivator and can help to give one's career a sense of purpose and meaning.\n\nOverall, altruism can be a powerful tool for building a successful and fulfilling career"""
5,"""What is imposter syndrome and how does it relate to AI?""","""\nImposter syndrome is a psychological condition where someone doubts their own abilities, talents, and accomplishments, even when they have evidence of their success. It is a common experience among people, with an estimated 70% of people experiencing it at some point. Imposter syndrome can be particularly prevalent in the field of AI, where individuals may feel like they don't have the necessary skills or knowledge to keep up with the rapidly evolving field. However, it's important to remember that imposter syndrome is a normal experience and doesn't necessarily reflect one's true abilities or accomplishments. It's important to acknowledge and address imposter syndrome in order to prevent it from hindering personal and professional growth in AI."""
6,"""Who are some accomplished individuals who have experienced imposter synd

In [25]:
tru.get_leaderboard(app_ids=["Sentence Window Engine 1"])

,latency,total_cost
app_id,,
Sentence Window Engine 1,15.454545,0.0


In [ ]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Provider List: https://docs.litellm.ai/docs/providers


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verb

<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 3. Sentence Window Size = 3

In [ ]:
Tru().reset_database()

tru_recorder_3 = get_prebuilt_trulens_recorder(
    sentence_window_engine_3,
    app_id='sentence window engine 3'
)

run_evals(eval_questions, tru_recorder_3, sentence_window_engine_3)

In [ ]:
Tru().get_leaderboard(app_ids=['sentence window engine 3'])

In [ ]:
Tru().run_dashboard()

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()